In [47]:
import cirq
import numpy as np
import random
import sympy

%matplotlib inline

np.random.seed(42)

In [68]:
def layer(length,W):
    for i in range(length):
        for j in range(length):
            rot1 = cirq.ZPowGate(exponent=W[i][j][0])
            rot2 = cirq.XPowGate(exponent=W[i][j][1])
            rot3 = cirq.ZPowGate(exponent=W[i][j][2])
            yield rot1(cirq.GridQubit(i, j))
            yield rot2(cirq.GridQubit(i, j))
            yield rot3(cirq.GridQubit(i, j))

rot = [[[0.1,0.2,0.3],[0.4,0.5,0.6]],
       [[0.7,0.8,0.9],[1.0,1.1,1.2]]]
       
circuit = cirq.Circuit()
circuit.append(layer(2,rot))
print(circuit)

(0, 0): ───Z^0.1───X^0.2────Z^0.3────

(0, 1): ───Z^0.4───X^0.5────Z^0.6────

(1, 0): ───Z^0.7───X^0.8────Z^0.9────

(1, 1): ───Z───────X^-0.9───Z^-0.8───


In [43]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""
    rot1 = cirq.ZPowGate(exponent=half_turns)
    rot2 = cirq.XPowGate(exponent=half_turns)
    rot3 = cirq.ZPowGate(exponent=half_turns)
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


In [25]:
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)
print('transverse fields: {}'.format(h))
print('row j fields: {}'.format(jr))
print('column j fields: {}'.format(jc))

transverse fields: [[1, 1, -1], [1, 1, 1], [1, -1, -1]]
row j fields: [[1, -1, -1], [-1, -1, -1]]
column j fields: [[1, 1], [1, -1], [1, 1]]


In [26]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i + 1, j))
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i, j + 1))
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))
                
def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    length = len(h)
    yield rot_x_layer(length, x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)

In [27]:
h, jr, jc = random_instance(3)

circuit = cirq.Circuit()    
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
               strategy=cirq.InsertStrategy.EARLIEST)
print(circuit)

                           ┌──────┐   ┌───────────┐               ┌───────────┐
(0, 0): ───X^0.1───X───────────────────@──────────────X───────X────@──────────────X───────────────────────────────────────────
                                       │                           │
(0, 1): ───X^0.1───Z^0.2────X──────────┼────@─────────X───────X────@^0.3──────────X───X───────@───────X───────────────────────
                                       │    │                                                 │
(0, 2): ───X^0.1───Z^0.2─────@─────────┼────┼────X────────────────────────────────────────────@^0.3───X───────────────────────
                             │         │    │
(1, 0): ───X^0.1───Z^0.2────X┼─────────@^0.3┼─────────X───────X────@──────────────X───@───────────────────────────────────────
                             │              │                      │                  │
(1, 1): ───X^0.1───Z^0.2────X┼──────────────@^0.3─────X───────X────┼────@─────────X───@^0.3───X───────@───────X──

In [28]:
simulator = cirq.Simulator()
circuit = cirq.Circuit()    
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({0: 78, 128: 4, 8: 4, 2: 3, 32: 3, 16: 2, 34: 2, 6: 1, 256: 1, 64: 1, 129: 1})


In [29]:

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy
print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

Counter({-7: 81, -3: 8, 1: 5, -1: 2, -5: 2, 3: 1, -9: 1})


In [31]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
print('Value of the objective function {}'.format(obj_func(results)))


Value of the objective function -6.04


In [35]:
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))
print(circuit)

                              ┌────────┐   ┌───────────────┐                 ┌───────────────┐
(0, 0): ───X^alpha───X──────────────────────@──────────────────X─────────X────@──────────────────X───────────────────────────────────────────────────M('x')───
                                            │                                 │                                                                      │
(0, 1): ───X^alpha───Z^beta────X────────────┼──────@───────────X─────────X────@^gamma────────────X───X─────────@─────────X───────────────────────────M────────
                                            │      │                                                           │                                     │
(0, 2): ───X^alpha───Z^beta─────@───────────┼──────┼──────X────────────────────────────────────────────────────@^gamma───X───────────────────────────M────────
                                │           │      │                                                                          

In [38]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)

sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
for result in results:
    print(result.params.param_dict, obj_func(result))

OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.1)]) -6.3
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.30000000000000004)]) -6.12
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.5)]) -6.08
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.7000000000000001)]) -6.42
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.9)]) -6.6
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.1)]) -6.24
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) -6.22
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.5)]) -6.36
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) -6.26
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.9)]) -6.32
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.1)]) -6.08
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.30000000000000004)]) -6.04
OrderedDict([('alpha', 0.1), ('beta', 0.

OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) -8.22
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.5)]) -7.86
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) -8.28
OrderedDict([('alpha', 0.9), ('beta', 0.30000000000000004), ('gamma', 0.9)]) -7.9
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.1)]) -7.96
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.30000000000000004)]) -8.54
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.5)]) -8.28
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.7000000000000001)]) -8.14
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.9)]) -8.22
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.1)]) -8.1
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.30000000000000004)]) -8.04
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.5)]) -8.42
OrderedDict

In [39]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=10))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print('Minimum objective value is {}.'.format(min))

Minimum objective value is -9.0.
